* DATASET: Apresentar dados que serão utilizados na soluçao.

* ANÁLISE: Planejar e gerar as análises que serão realizadas com os dados propostos em código python.

* INSIGHT: Apresentar resultados (em forma de listagem e/ou gráfico) e gerar insights relacionados a solução proposta. Os insights deverão ser disponibilizados como comentários no arquivo, logo após o resultado. Considere que os insights precisam ser bem compreendidos. Portanto, explique bem seus insights.
 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
df_ota_bus_ticket_sales = spark.table("bus.bronze.ota_bus_ticket_sales")

df_ota_bus_ticket_sales = df_ota_bus_ticket_sales.withColumn("year_purchase", year(col("date_purchase"))) \
                   .withColumn("month_purchase", month(col("date_purchase"))) \
                   .withColumn("day_of_week_purchase", dayofweek(col("date_purchase")))

In [0]:
print("Informações gerais:")
df_ota_bus_ticket_sales.printSchema()

In [0]:
print("\nEstatísticas descritivas:")
display(df_ota_bus_ticket_sales.describe())

In [0]:
df_pandas = df_ota_bus_ticket_sales.select("gmv_success").toPandas()
plt.figure(figsize=(10, 5))
sns.histplot(df_pandas["gmv_success"])
plt.xlabel("Valor das compras R$")
plt.ylabel("Frequência de compras")
plt.title("Distribuição dos valores de compras")
display(plt.gcf())
plt.clf()

In [0]:
df_pandas = df_ota_bus_ticket_sales.select("gmv_success").toPandas()
plt.figure(figsize=(10, 5))
sns.histplot(df_pandas["gmv_success"])
plt.xlabel("Valor das compras R$")
plt.ylabel("Frequência de compras")
plt.title("Distribuição dos valores de compras")
display(plt.gcf())
plt.clf()

# EIXO X VAI SER O PERÍODO PARA ANALISAR A DISTRIBUIÇÃO DE GASTOS HISTÓRICOS

In [0]:
top_destinos = df_ota_bus_ticket_sales.groupBy("place_destination_departure").count().orderBy(col("count").desc()).limit(10)
top_destinos_pandas = top_destinos.toPandas()
plt.figure(figsize=(10, 5))
sns.barplot(x=top_destinos_pandas["count"], y=top_destinos_pandas["place_destination_departure"])
plt.title("Top 10 destinos mais procurados")
display(plt.gcf())
plt.clf()

In [0]:
%sql
select
  date_purchase,
  place_destination_departure,
  count(distinct nk_ota_localizer_id) as qtd,
  sum(gmv_success) as gmv_success
from
  bus.gold.ota_bus_ticket_sales
group by
  all
order by
  qtd desc;


Databricks visualization. Run in Databricks to view.

In [0]:
top_destinos = df_ota_bus_ticket_sales.groupBy("place_destination_departure").count().orderBy(col("count").desc()).limit(10)
top_destinos_pandas = top_destinos.toPandas()
plt.figure(figsize=(10, 5))
sns.barplot(x=top_destinos_pandas["count"], y=top_destinos_pandas["place_destination_departure"])
plt.title("Top 10 destinos mais procurados")
display(plt.gcf())
plt.clf()

In [0]:
from pyspark.sql.functions import max, count, col, lit, datediff

clientes = df_ota_bus_ticket_sales.groupBy("fk_contact").agg(
    max("gmv_success").alias("total_gasto"),
    count("nk_ota_localizer_id").alias("qtd_compras"),
    max("date_purchase").alias("ultima_compra")
)

max_date = df_ota_bus_ticket_sales.select(
    max("date_purchase")
).collect()[0][0]

clientes = clientes.withColumn(
    "recencia",
    datediff(
        lit(max_date),
        col("ultima_compra")
    )
)

rfm = clientes.select(
    "recencia",
    "qtd_compras",
    "total_gasto"
).toPandas()

rfm_norm = (rfm - rfm.min()) / (rfm.max() - rfm.min())

In [0]:
kmeans = KMeans(n_clusters=4, random_state=42)
rfm["cluster"] = kmeans.fit_predict(rfm_norm)

plt.figure(figsize=(10, 5))
sns.scatterplot(data=rfm, x="qtd_compras", y="total_gasto", hue="cluster", palette="tab10")
plt.title("Clusterização de Clientes (Frequência x Gasto)")
display(plt.gcf())
plt.clf()

rfm["fk_contact"] = clientes.select("fk_contact").toPandas()
print("\nResumo dos clusters:")
rfm_grouped = rfm.groupby("cluster").agg({
    "qtd_compras": "mean",
    "total_gasto": "mean",
    "recencia": "mean",
    "fk_contact": "count"
})
display(rfm_grouped)